In [24]:
import requests
import bs4
import openai
import os

In [6]:
country = input("What country are you interested in for news?")
country

What country are you interested in for news?France


'France'

In [8]:
country_newspapers = {"Spain":"https://elpais.com",
                     "France":"https://lemonde.fr"}
country_newspapers[country]

'https://lemonde.fr'

In [42]:
url = country_newspapers[country]
result = requests.get(url)
#result.text

In [15]:
tag = 'article__title-label'
soup = bs4.BeautifulSoup(result.text,'lxml')


for item in soup.select('.article__title')[:3]:
    print(item.getText())

Transports : l’Allemagne lance l’abonnement unique à 49 euros
Jeux vidéo : le Royaume-Uni se met en travers de la fusion de Microsoft et Activision
Matières premières : « L’or sort de l’ordinaire par rapport aux autres commodités »


In [17]:
# Restate the newspaper target
country_newspapers = {"Spain":("https://elpais.com",'.c_t'),
                     "France":("https://lemonde.fr/','.article__title-label")}

In [39]:
# Function to take in information for the prompt to OpenAI

def create_prompt():
    country = input("Choose country for news summary")
    try:
        url, tag = country_newspapers[country]
    except:
        print('Country not supported')
        return
    
    results = requests.get(url)
    soup = bs4.BeautifulSoup(results.text,'lxml')
    
    country_headlines = ''
    for item in soup.select(tag)[:3]:
        country_headlines += item.getText()+'\n'
               
    prompt = "Detect the language of the news headlines below, then translate a summary of the headlines to English in a conversational tone.\n"
    return prompt + country_headlines


In [40]:
prompt = create_prompt()
prompt

Choose country for news summarySpain


'Detect the language of the news headlines below, then translate a summary of the headlines to English in a conversational tone.\nDiario de un cura pederasta: “Hice daño a demasiados”\nLas huelgas salariales agravan la crisis de la justicia\n\nEl avispero de los regadíos de\xa0Doñana\n'

In [41]:
print(prompt)

Detect the language of the news headlines below, then translate a summary of the headlines to English in a conversational tone.
Diario de un cura pederasta: “Hice daño a demasiados”
Las huelgas salariales agravan la crisis de la justicia

El avispero de los regadíos de Doñana



In [31]:
#os.environ['OPEN_AI_KEY'] = ''

In [34]:
openai.api_key = os.getenv('OPEN_AI_KEY')

In [37]:
response = openai.Completion.create(model = 'text-davinci-003',
                                   prompt = prompt,
                                   max_tokens = 200,
                                   temperature = 0.1,
                                   top_p = 1.0
                                   
                                   )

In [38]:
print(response['choices'][0]['text'])


Language: Spanish

Translation: A priest's diary reveals he caused harm to many people. Wage strikes are worsening the justice crisis. The Doñana irrigation system is causing a stir.
